In [2]:
import tensorflow as tf

import requests
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

## Load in the data

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# the data is only 2D
# convolution expects height x width x color
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


## Build the model using functional API

In [7]:
i = Input(shape=x_train[0].shape)
x = Conv2D(filters=32, kernel_size=(3,3), strides=2, activation='relu')(i)
x = Conv2D(filters=64, kernel_size=(3,3), strides=2, activation='relu')(x)
x = Conv2D(filters=128, kernel_size=(3,3), strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=K, activation='softmax')(x)

model = Model(i, x)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656

In [9]:
# compile and fit
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 20s 10ms/step - loss: 0.5149 - accuracy: 0.8085 - val_loss: 0.3890 - val_accuracy: 0.8511
Epoch 2/15
1875/1875 [==============================] - 18s 9ms/step - loss: 0.3527 - accuracy: 0.8679 - val_loss: 0.3531 - val_accuracy: 0.8678
Epoch 3/15
1875/1875 [==============================] - 18s 10ms/step - loss: 0.3061 - accuracy: 0.8862 - val_loss: 0.3163 - val_accuracy: 0.8853
Epoch 4/15
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2734 - accuracy: 0.8960 - val_loss: 0.3128 - val_accuracy: 0.8853
Epoch 5/15
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2487 - accuracy: 0.9065 - val_loss: 0.2916 - val_accuracy: 0.8968
Epoch 6/15
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2253 - accuracy: 0.9158 - val_loss: 0.2866 - val_accuracy: 0.8985
Epoch 7/15
1875/1875 [==============================] - 16s 8ms/step - loss: 0.2089 - accuracy: 0.9215 - val_loss: 0.2925 - val

## Save model to a temp directory 

In [11]:
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}
    
tf.saved_model.save(model, export_path)

print('\nSaved model:')
!dir {export_path}

export_path = C:\Users\mainn.ho\AppData\Local\Temp\1


Already saved a model, cleaning up



'rm' is not recognized as an internal or external command,
operable program or batch file.


INFO:tensorflow:Assets written to: C:\Users\mainn.ho\AppData\Local\Temp\1\assets


INFO:tensorflow:Assets written to: C:\Users\mainn.ho\AppData\Local\Temp\1\assets



Saved model:
 Volume in drive C has no label.
 Volume Serial Number is 52F9-EE60

 Directory of C:\Users\mainn.ho\AppData\Local\Temp\1

14/08/2024  10:17 SA    <DIR>          .
14/08/2024  10:17 SA    <DIR>          ..
14/08/2024  10:17 SA    <DIR>          assets
14/08/2024  10:17 SA                55 fingerprint.pb
14/08/2024  10:17 SA           183,294 saved_model.pb
14/08/2024  10:17 SA    <DIR>          variables
               2 File(s)        183,349 bytes
               4 Dir(s)   6,133,784,576 bytes free


In [12]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
The MetaGraph with tag set ['serve'] contains the following ops: {'MergeV2Checkpoints', 'ShardedFilename', 'NoOp', 'AssignVariableO

In [14]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

# !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
# !apt update

# Run the echo command and add the repository
subprocess.run("wsl echo 'deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal' | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list", shell=True)

# Run the curl command to add the GPG key
subprocess.run("wsl curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -", shell=True)

# Run apt update
subprocess.run("wsl sudo apt update", shell=True)

CompletedProcess(args='wsl sudo apt update', returncode=1)

In [16]:
# !apt-get install tensorflow-model-server

# Command to install tensorflow-model-server using apt-get
command = "sudo apt-get install -y tensorflow-model-server"

# Running the command via WSL
subprocess.run(f"wsl {command}", shell=True, check=True)


CalledProcessError: Command 'wsl sudo apt-get install -y tensorflow-model-server' returned non-zero exit status 1.